3.13 丢弃法
除了前一节介绍的权重衰减以外，深度学习模型常常使用丢弃法（dropout）[1] 来应对过拟合问题。丢弃法有一些不同的变体。本节中提到的丢弃法特指倒置丢弃法（inverted dropout）


根据丢弃法的定义，我们可以很容易地实现它。下面的dropout函数将以drop_prob的概率丢弃NDArray输入X中的元素。

In [26]:
import tensorflow as tf
import numpy as np
from tensorflow import keras,nn
from tensorflow.keras.layers import Dropout,Flatten,Dense
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"   
def dropout(X, drop_prob):
    assert 0 <= drop_prob <= 1
    keep_prob = 1 - drop_prob
    # 这种情况下把全部元素都丢弃
    if keep_prob == 0:
        return tf.zeros_like(X)
    #初始mask为一个bool型数组，故需要强制类型转换
    mask = tf.keras.backend.random_uniform( X.shape,0,1,dtype=tf.float32)<keep_prob
    mask=tf.cast(mask,tf.float32)
#     print(mask)
    return mask * X / keep_prob

In [27]:
X = tf.range(0,16,dtype='float32')
X=tf.reshape(X,[2,8])
print(X)
dropout(X, 0)

tf.Tensor(
[[ 0.  1.  2.  3.  4.  5.  6.  7.]
 [ 8.  9. 10. 11. 12. 13. 14. 15.]], shape=(2, 8), dtype=float32)


<tf.Tensor: id=3464671, shape=(2, 8), dtype=float32, numpy=
array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11., 12., 13., 14., 15.]], dtype=float32)>

In [28]:
dropout(X, 0.5)

<tf.Tensor: id=3464684, shape=(2, 8), dtype=float32, numpy=
array([[ 0.,  2.,  4.,  0.,  8., 10.,  0.,  0.],
       [16., 18.,  0., 22., 24.,  0.,  0., 30.]], dtype=float32)>

In [29]:
dropout(X, 1)

<tf.Tensor: id=3464685, shape=(2, 8), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

3.13.2.1. 定义模型参数¶
实验中，我们依然使用“softmax回归的从零开始实现”一节中介绍的Fashion-MNIST数据集。我们将定义一个包含两个隐藏层的多层感知机，其中两个隐藏层的输出个数都是256。

In [30]:
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10, 256, 256
batch_size=15
(x_train,y_train),(x_test,y_test)=keras.datasets.mnist.load_data()
x_train = tf.cast(x_train, tf.float32)
x_test=tf.cast(x_test,tf.float32)
train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
# iter用来生成迭代器
train_iter = iter(train_db)
# next() 返回迭代器的下一个项目
sample = next(train_iter)
w1 = tf.Variable(tf.random.truncated_normal(shape=(num_inputs, num_hiddens1), stddev=0.1))
b1 = tf.Variable(tf.random.truncated_normal([num_hiddens1], stddev=0.1))
w2 = tf.Variable(tf.random.truncated_normal(shape=(num_hiddens1, num_hiddens2), stddev=0.1))
b2=tf.Variable(tf.random.truncated_normal([num_hiddens2], stddev=0.1))
w3 = tf.Variable(tf.random.truncated_normal(shape=(num_hiddens2, num_outputs), stddev=0.1))
b3=tf.Variable(tf.random.truncated_normal([num_outputs], stddev=0.1))
# params = [W1, b1, W2, b2, W3, b3]

3.13.2.2. 定义模型
下面定义的模型将全连接层和激活函数ReLU串起来，并对每个激活函数的输出使用丢弃法。我们可以分别设置各个层的丢弃概率。通常的建议是把靠近输入层的丢弃概率设得小一点。在这个实验中，我们把第一个隐藏层的丢弃概率设为0.2，把第二个隐藏层的丢弃概率设为0.5。我们可以通过“自动求梯度”一节中介绍的is_training函数来判断运行模式为训练还是测试，并只需在训练模式下使用丢弃法。

In [31]:
def net(X):
    X =tf.reshape(X,(-1,num_inputs))
#     print(X.shape,w1.shape)
    H1 = nn.relu((tf.matmul(X, w1) + b1))
    # 只在训练模型时使用丢弃法
    H1 = dropout(H1, drop_prob1)  # 在第一层全连接后添加丢弃层
    H2 = nn.relu((tf.matmul(H1,w2)+b2))
    H2 = dropout(H2, drop_prob2)  # 在第二层全连接后添加丢弃层
    return tf.math.softmax( tf.matmul(H2,w3) + b3 )

In [32]:
num_epochs, lr, batch_size = 5, 0.5, 256
drop_prob1, drop_prob2 = 0.2, 0.5
loss_all = 0
for epoch in range(num_epochs):  #数据集级别迭代
    for step,(x_train,y_train) in enumerate(train_db):  #batch级别迭代
        with tf.GradientTape() as tape:
#             print(x_train.shape)
            y_hat = net(x_train)
            loss =tf.reduce_mean(tf.losses.sparse_categorical_crossentropy(y_train,y_hat))#采用交叉熵损失函数
            loss_all += loss.numpy()#将loss的张量转化为numpy型用来相加

            # compute gradients 进行梯度的计算
            grads = tape.gradient(loss, [w1, b1, w2, b2,w3,b3])
            w1.assign_sub(grads[0])
            b1.assign_sub(grads[1])
            w2.assign_sub(grads[2])
            b2.assign_sub(grads[3])
            w3.assign_sub(grads[4])
            b3.assign_sub(grads[5])

    print(epoch, 'loss:', loss.numpy())
    total_correct, total_number = 0, 0

    for x,y in test_db :
        with tf.GradientTape() as tape:
            H1 = nn.relu((tf.matmul(x, w1) + b1))
            # 只在训练模型时使用丢弃法
            H2 = nn.relu((tf.matmul(H1,w2)+b2))
            y_hat = tf.math.softmax( tf.matmul(H2,w3) + b3 )
            y=tf.cast(y,'int64')
            correct=acc(y_hat,y)
    print(epoch,"test_acc:", correct)

0 loss: 15.043557


InvalidArgumentError: In[0] mismatch In[1] shape: 28 vs. 784: [15,28,28] [784,256] 0 0 [Op:BatchMatMulV2] name: MatMul/

3.13.3 简洁实现
在Tensorflow2.0中，我们只需要在全连接层后添加Dropout层并指定丢弃概率。在训练模型时，Dropout层将以指定的丢弃概率随机丢弃上一层的输出元素；在测试模型时（即model.eval()后），Dropout层并不发挥作用。

In [36]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(256,activation='relu'),
    Dropout(0.2),
    keras.layers.Dense(256,activation='relu'),
    Dropout(0.5),
    keras.layers.Dense(10,activation=tf.nn.softmax)
])
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train,y_train,epochs=5,batch_size=256,validation_data=(x_test, y_test),
                    validation_freq=1)


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 2s 39us/sample - loss: 7.0662 - accuracy: 0.7214 - val_loss: 0.6324 - val_accuracy: 0.8947
Epoch 2/5
60000/60000 [==============================] - 2s 30us/sample - loss: 0.9542 - accuracy: 0.8231 - val_loss: 0.4979 - val_accuracy: 0.9069
Epoch 3/5
60000/60000 [==============================] - 2s 31us/sample - loss: 0.6399 - accuracy: 0.8674 - val_loss: 0.3951 - val_accuracy: 0.9231
Epoch 4/5
60000/60000 [==============================] - 2s 30us/sample - loss: 0.4849 - accuracy: 0.8937 - val_loss: 0.3139 - val_accuracy: 0.9328
Epoch 5/5
60000/60000 [==============================] - 2s 30us/sample - loss: 0.4048 - accuracy: 0.9066 - val_loss: 0.2716 - val_accuracy: 0.9424


小结
我们可以通过使用丢弃法应对过拟合。
丢弃法只在训练模型时使用。


